In [1]:
# Libraries
import numpy as np
import pandas as pd

In [2]:
# Function to invoke Schur program and perform products (λ1,μ1)x(λ2,μ2) 
# s is an optional argument to distinguish between input filenames in parallel computations

def SU3prod(l1, m1, l2, m2, s=""):
    
    l1 = int(l1)
    l2 = int(l2)
    m1 = int(m1)
    m2 = int(m2)

    # Formatting parameters to Schur program
    if (l1+m1 >= 10):
        arg1 = "!"+str(l1+m1)
    else:
        arg1 = str(l1+m1)
    #...................................................    
    if (m1 >= 10):
        arg3 = "!"+str(m1)
    else:
        arg3 = str(m1)
        
    #...................................................    
    if (l2+m2 >= 10):
        arg2 = "!"+str(l2+m2)
    else: 
        arg2 = str(l2+m2)
    
    #...................................................    
    if (m2 >= 10):
        arg4 = "!"+str(m2)
    else: 
        arg4 = str(m2)
    
    #...................................................    
    prod = f"prod {{{arg1} {arg3}}},{{{arg2} {arg4}}}\n"
    
    # Input file to deliver to Schur
    with open('schurinput'+s+'.txt','w') as f:
        f.write("rep\n")
        f.write("gr1 su3\n")
        f.write(prod)
        f.write("^C")
    f.close()

    # Run Schur
    STR = !schur < schurinput{s}.txt
    ! rm schurinput{s}.txt
    
    # Extract the irreps
    index = 0
    for i in STR:
        if i == "     ^":
            break
        index += 1
        
    irreps = ""
    for i in STR[23:index]:
        irreps += i
        
    # Separate each irrep
    irrepsSep = irreps.split(" + ")
            
    irrepsArr = []
    for i in irrepsSep:
        
        openkey = i.rfind("{")
        closekey = i.rfind("}")
        
        # Extract multiplicites
        mult = 1
        if openkey != 0:
            mult = int(i[0:openkey])
        
        l_m = i[openkey+1:closekey].split(" ")
                    
        # Extract (λ1,μ1) values
        if (len(l_m)==3):
            irrepsArr.append([mult, int(l_m[0])-int(l_m[1]), int(l_m[1])])
        
        if (len(l_m)==2):
            if (l_m[1]==""):
                irrepsArr.append([mult, int(l_m[0]), 0])
            elif (l_m[1] == "^2"):
                irrepsArr.append([mult, 0, int(l_m[0])])
            else:
                irrepsArr.append([mult, int(l_m[0])-int(l_m[1]), int(l_m[1])])

        if (len(l_m)==1):
            if (l_m[0].rfind("^") == -1):
                if (len(l_m[0])==2):
                    irrepsArr.append([mult, int(l_m[0][0])-int(l_m[0][1]), int(l_m[0][1])])
                if (len(l_m[0])==1):
                    irrepsArr.append([mult, int(l_m[0][0]), 0])
            else: 
                p = l_m[0].rfind("^")
                irrepsArr.append([mult, 0, int(l_m[0][0:p])])
                    
    values_df = pd.DataFrame(np.array(irrepsArr), columns = ["mult","lam","mu"])

    return values_df

In [3]:
# Function to invoke Schur program and compute dimension of irrep (λ,μ)
# s is an optional argument to distinguish between input filenames in parallel computations

def SU3dim(l, m, s=""):
    
    l = int(l)
    m = int(m)
    
    # Formatting parameters to Schur program
    if (l+m >= 10):
        arg1 = "!"+str(l+m)
    else:
        arg1 = str(l+m)
    #...................................................    
    if (m >= 10):
        arg2 = "!"+str(m)
    else:
        arg2 = str(m)
    
    #...................................................    
    dim = f"dim {{{arg1} {arg2}}}\n"
    
    # Input file to deliver to Schur
    with open('schurinputdim'+s+'.txt','w') as f:
        f.write("rep\n")
        f.write("gr1 su3\n")
        f.write(dim)
        f.write("^C")
    f.close()

    # Run Schur
    STR = !schur < schurinputdim{s}.txt
    ! rm schurinputdim{s}.txt
        
    # Extract the dimension line
    index = 0
    for i in STR:
        if i == "     ^":
            break
        index += 1
        
    dimension = int(STR[23:index][0].split(" = ")[1])

    return dimension

In [4]:
# Function to invoke su3lib and compute Wigner coefficients
# s is an optional argument to distinguish between input filenames in parallel computations

def SU3WignerCoeff(l1, m1, L1, l2, m2, L2, l3, m3, L3, s=""):
    
    # Input file to deliver to su3lib
    with open('su3wignercoeffs'+s+'.txt','w') as f:
        f.write(f"{int(l1)} {int(m1)}\n")
        f.write(f"{int(L1)}\n")
        f.write(f"{int(l2)} {int(m2)}\n")
        f.write(f"{int(L2)}\n")        
        f.write(f"{int(l3)} {int(m3)}\n")
        f.write(f"{int(L3)}\n")
    f.close()
    
    # Run SU3_SO3_WignerCoeffs
    lines = !/home/alejandro/Downloads/Thesis_Programs/su3libCorrection/su3lib/tools/SU3_SO3_WignerCoeffs < su3wignercoeffs{s}.txt
    !rm su3wignercoeffs{s}.txt
    
    # Cases where the coupling is allowed
    if len(lines) > 8:
        
        # Values and labels stored
        fl = list(filter(lambda x: x != '', lines[8].split(" ")))
        flv = [fl[2], fl[3], fl[6], fl[7], fl[10:]]
        values = [flv[:4] + flv[4]]
        values = values + [list(filter(lambda x: x != '', l.split(" "))) for l in lines[9:]]

        # Column names and multiplicities
        cols = ["k1", "L1", "k2", "L2", "k3", "L3"]
        rhos = []
        for i in range(1,len(values[0])-5):
            cols.append("rho{0}".format(i))
            rhos.append("rho{0}".format(i))
            
        # Values stored
        values_df = pd.DataFrame(np.array(values), columns = cols)
        values_df[["k1", "L1", "k2", "L2", "k3", "L3"]] = values_df[["k1", "L1", "k2", "L2", "k3", "L3"]].astype(int)
        values_df[rhos] = values_df[rhos].astype(float)
        
    else:
        cols = ["k1", "L1", "k2", "L2", "k3", "L3"]
        values_df = pd.DataFrame(columns = cols)

    return values_df

In [5]:
# Function to invoke su3lib and compute 9-(λ,μ)
# s is an optional argument to distinguish between input filenames in parallel computations

def SU39lammu(l1 , m1 , l2 , m2 , l12, m12,
              l3 , m3 , l4 , m4 , l34, m34,
              l13, m13, l24, m24, l  , m  , s=""):
    
    # Input file to deliver to su3lib
    with open('su39lammu'+s+'.txt','w') as f:
        f.write(f"{int(l1)} {int(m1)}\n")
        f.write(f"{int(l2)} {int(m2)}\n")
        f.write(f"{int(l12)} {int(m12)}\n")
        f.write(f"{int(l3)} {int(m3)}\n")
        f.write(f"{int(l4)} {int(m4)}\n")
        f.write(f"{int(l34)} {int(m34)}\n")
        f.write(f"{int(l13)} {int(m13)}\n")
        f.write(f"{int(l24)} {int(m24)}\n")
        f.write(f"{int(l)} {int(m)}\n")
    f.close()
    
    # Run SU3_U9Coeffs
    lines = !/home/alejandro/Downloads/Thesis_Programs/su3libCorrection/su3lib/tools/SU3_U9Coeffs < su39lammu{s}.txt
    !rm su39lammu{s}.txt
        
    # Cases where the coupling is allowed
    if len(lines) > 20:

        # Column names of the multiplicities
        cols = list(filter(lambda x: x != '', lines[20].split(" ")))
        cols.append("value")
        cols[5] = cols[5][:5]

        # Multiplicities values stored
        values = [list(filter(lambda x: x != '', l.split(" "))) for l in lines[21:]]
        values_df = pd.DataFrame(np.array(values), columns = cols)
        values_df["value"] = values_df["value"].astype(float)
        values_df[["rho1324","rho24", "rho13", "rho1234", "rho34", "rho12"]] = values_df[["rho1324","rho24", "rho13", "rho1234", "rho34", "rho12"]].astype(int)

    else:
        values_df = pd.DataFrame()

    return values_df

In [6]:
# Function to invoke abljint and obtain the SU(3) irreps contained in U(N) 
# s is an optional argument to distinguish between input filenames in parallel computations

def UNtoU3(eta, UNirrep, s=""):
    
    eta = int(eta)
    
    # Input file to deliver to su3lib
    with open('UNtoU3'+s+'.txt','w') as f:
        f.write("1\n")
        f.write(f"{eta}\n")
        
        irr = ""
        for i in range(0, (eta+1)*(eta+2)//2):
            try:
                irr += str(UNirrep[i]) + " "
            except:
                irr += "0 "    
        
        f.write(f"{irr}\n")
        f.write("^C")
    f.close()

    # Run UNtoU3 algorithm
    lines = !/home/alejandro/Downloads/Thesis_Programs/Fortran_H.O/abljint < UNtoU3{s}.txt
    !rm UNtoU3{s}.txt
    
    # Extract the lines
    beginreps = lines.index("0            ( LM, MU)       NUM      DIM       C2       C3")
    endreps = lines[1:].index("1 DO YOU WANT DETAILED OUTPUT? ... YES (0) OR NO (1)")

    # Values and labels stored
    vv = [list(filter(lambda x: x != '', l.split(" "))) for l in lines[beginreps+1:endreps+1]]
    values = []
    for i in vv:
        
        # Append the multiplicities of U3 contained in UN 
        for mult in range(1, int(i[3])+1):
            values.append([int(i[1].rstrip(i[1][-1])), int(i[2].rstrip(i[2][-1])), mult, int(i[4]), int(i[5]), int(i[6])])

    # Values stored in dataframe
    values_df = pd.DataFrame(np.array(values), columns = ["lam","mu","mult", "dim", "C2", "C3"])

    return values_df

In [7]:
# Function to compute the conjugate of a given irrep f. f must be a list

def Conjugate(f):
    
    # Copy of irrep and array to compute the conjugate
    copy_f = f.copy()
    conj = []
    
    # Compute the conjugate irrp by substracting the one to each row  
    while (copy_f != list(np.array(np.zeros((len(copy_f)))))):
    
        # Elements in the column
        col = 0
        
        # Counts elements in column and substract to the irrep
        for i in range(len(copy_f)):

            if copy_f[i] != 0:
                copy_f[i] = copy_f[i]-1
                col += 1

        conj.append(col)
    
    # Complete the labels if necessary
    if len(conj) == 1:
        conj.append(0)
        
    if len(conj) == 0:
        conj.append(0)
        conj.append(0)
        
    return conj

In [8]:
# Function to compute rme of creation operators in proton-neutron scheme
# bra: [[f'], λ', μ'] 
# ket: [[f], λ, μ] 
# eta: η
# s is an optional argument to distinguish between input filenames in parallel computations

def SU3rme(bra, ket, eta, s=""):
    
    # Extract the labels
    f, l, m = ket
    fq, lq, mq = bra  
    
    # Conjugate spin irrep
    fbar  = Conjugate(f)
    fbarq = Conjugate(fq)
    
    #...............................................................................................
    # Input file to deliver to hwsgen code
    with open('hwsir.py'+s,'w') as f:
        f.write("oscillator shell number (eta)\n")
        f.write(f"{int(eta)}\n")
        f.write(f"lm mu f1 f2 lm mu fl f1\n")
        f.write(f"{int(lq)} {int(mq)} {fbarq[0]} {fbarq[1]} {int(l)} {int(m)} {fbar[0]} {fbar[1]}")
    f.close()

    # Input file to run hwsgen code
    with open('hwsgen'+s+'.txt','w') as f:
        f.write("15\n")
        f.write("py"+s+"\n")
        f.write("0\n")
        f.write("1\n")
        f.write("2\n")
    f.close()
    
    # Run hwsgen. The executable must be located at the same route than this notebook
    linesHWS = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/hwsgen.out < hwsgen{s}.txt
    !rm hwsgen{s}.txt

    #...............................................................................................
    # Input file to run opbgen code. The executable must be located at the same route than this notebook
    with open('opbgen'+s+'.txt','w') as f:
        f.write("py"+s+"\n")
        f.write("0\n")
        f.write(f"{int(eta)}\n")
        f.write(f"1 0\n")
    f.close()
    
    # Run opbgen
    linesOPB = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/opbgen.out < opbgen{s}.txt
    !rm opbgen{s}.txt

    #....................................................................................
    # Input file to run rmegen code. The executable must be located at the same route than this notebook
    with open('rmegen'+s+'.txt','w') as f:
        f.write("15\n")
        f.write("py"+s+"\n")
        f.write("0\n")
        f.write("2\n")
        f.write("3\n")
        f.write("1 0\n")
        f.write("3\n")
        f.write("0\n")
        f.write("0 0\n")
        f.write("1000 1000\n")
        f.write("0\n")
    f.close()
    
    # Run rmegen
    linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    
    #  Unexpected error XEWU3 DIMENSION OVERFLOW, run again rmegen.out. Optimize this section !!!!!!!!   
    for i in linesRME:
        if " ***** XEWU3 DIMENSION OVERFLOW:" in i:
            print(" ***** XEWU3 DIMENSION OVERFLOW ==============================================================")
            linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    # In case it appears a second consecutive time 
    for i in linesRME:
        if " ***** XEWU3 DIMENSION OVERFLOW:" in i:
            print(" ***** XEWU3 DIMENSION OVERFLOW ==============================================================")
            linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    # In case it appears a third consecutive time 
    for i in linesRME:
        if " ***** XEWU3 DIMENSION OVERFLOW:" in i:
            print(" ***** XEWU3 DIMENSION OVERFLOW ==============================================================")
            linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    # In case it appears a fourth consecutive time 
    for i in linesRME:
        if " ***** XEWU3 DIMENSION OVERFLOW:" in i:
            print(" ***** XEWU3 DIMENSION OVERFLOW ==============================================================")
            linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    # In case it appears a third consecutive time 
    for i in linesRME:
        if " ***** XEWU3 DIMENSION OVERFLOW:" in i:
            print(" ***** XEWU3 DIMENSION OVERFLOW ==============================================================")
            linesRME = !/home/alejandro/Downloads/Octupole_deformation/r3Y3_operator/r3Y3_operator/rmegen.out < rmegen{s}.txt
    
    !rm rmegen{s}.txt

    # Delete intermediate files
    !rm *py{s}

    # Extract the line with the matrix element value
    try:
        index = 0
        for i in linesRME:
            if i == "    < bra ||| N1 (LM MU)ST(x)N2 (LM MU)ST;RO( LM MU)ST ||| ket >RO":
                break
            if i == " ***** ATTENTION  RME: Trivial solution.":
                print("Trivial solution")
                return 1
            if i == " ***** ATTENTION  RME: Number of particles not match.":
                print("Number of particles not match")
                return 0
            index += 1
            
        return float(linesRME[index+1].split("     ")[6])
    except:        
        print(linesRME)
        print("Unable to calculate matrix element <...||| {a+a}... |||...>.")

In [9]:
# Function to invoke Schur program and compute dimension of irrep [f] of U(N)
# s is an optional argument to distinguish between input filenames in parallel computations

def UNdim(eta, UNirrep, s=""):
    
    eta = int(eta)
    Omega = int((eta+1)*(eta+2)/2)
    
    # Prepare the input for the code 
    dim = "dim {"
    for i in UNirrep:
        dim = dim + str(i) + " "
    dim = dim + "}\n"
            
    # Input file to deliver to Schur
    with open('schurUNdim'+s+'.txt','w') as f:
        f.write("rep\n")
        f.write(f"gr1 u{Omega}\n")
        f.write(dim)
        f.write("^C")
    f.close()

    # Run Schur
    STR = !schur < schurUNdim{s}.txt
    ! rm schurUNdim{s}.txt
        
    # Extract the dimension line
    index = 0
    for i in STR:
        if i == "     ^":
            break
        index += 1
        
    dimension = int(STR[23:index][0].split(" = ")[1])

    return dimension